In [48]:
import os, json, time, requests
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime

# -------------------------------
# Settings laden / default erstellen
# -------------------------------
settings_file = "settings.json"
if not os.path.exists(settings_file):
    default_settings = {
        "allgemein_settings": {
            "model_folder": "models",
            "use_model_file": "Heusc_v0.3_20250913_024701.keras",
            "sequence_length": 60,
            "train_epochs": 10000,
            "batch_size": 32,
            "validation_split": 0.2,
            "early_stopping_patience": 5,
            "feature_scaling": True,
            "use_dropout": True,
            "dropout_rate": 0.2,
            "loss_function": "binary_crossentropy",
            "optimizer": "adam",
            "debug_mode": True
        },
        "offline": {
            "enabled": True,
            "csv_folder": "csv/binance",
            "csv_file": "BTCUSDT-1m-1y-binance-2025-09-12_22-40-01.csv"
        },
        "online": {
            "symbols": ["BTC","ETH","SOL"],
            "currencies": ["USDT","USDT","USDT"],
            "interval": "1m",
            "poll_seconds": 10,
            "max_live_train_minutes": 60
        },
        "balance": {
            "use_simulation": True,
            "initial_balance": 1000,
            "save_on_profit_percent": 50,
            "game_over_threshold": 0,
            "balance_reward_factor": 1.0,
            "retrain_from_best": True
        },
        "training_options": {
            "use_weighting": True,
            "punish_on_wrong": True,
            "predict_confidence": True,
            "log_transactions": True,
            "continual_learning": True,
            "transfer_learning": True,
            "ensemble_enabled": True
        }
    }
    with open(settings_file,"w") as f:
        json.dump(default_settings, f, indent=4)

with open(settings_file,"r") as f:
    settings = json.load(f)

allg = settings['allgemein_settings']
online = settings['online']
offline = settings['offline']
balance_settings = settings['balance']
training_options = settings['training_options']

print(f"allg: {allg}")
print(f"online: {online}")
print(f"offline: {offline}")
print(f"balance_settings: {balance_settings}")
print(f"training_opts: {training_opts}")

allg: {'model_folder': 'models', 'use_model_file': '', 'sequence_length': 60, 'train_epochs': 10000, 'batch_size': 32, 'validation_split': 0.2, 'early_stopping_patience': 5, 'feature_scaling': True, 'use_dropout': True, 'dropout_rate': 0.2, 'loss_function': 'binary_crossentropy', 'optimizer': 'adam', 'debug_mode': True}
online: {'symbols': ['BTC', 'ETH', 'SOL'], 'currencies': ['USDT', 'USDT', 'USDT'], 'interval': '1m', 'poll_seconds': 10, 'max_live_train_minutes': 60}
offline: {'enabled': True, 'csv_folder': 'csv/binance', 'csv_file': 'BTCUSDT-1m-1y-binance-2025-09-12_22-40-01.csv'}
balance_settings: {'use_simulation': True, 'initial_balance': 1000, 'save_on_profit_percent': 50, 'game_over_threshold': 0, 'balance_reward_factor': 1.0, 'retrain_from_best': True}
training_opts: {'use_weighting': True, 'punish_on_wrong': True, 'predict_confidence': True, 'log_transactions': True}


In [49]:
# -------------------------------
# Modell erstellen oder laden
# -------------------------------
def create_model(input_dim=7):
    model = Sequential([
        Dense(64, activation="relu", input_shape=(input_dim,)),
        Dropout(allg.get("dropout_rate",0.2)),
        Dense(32, activation="relu"),
        Dropout(allg.get("dropout_rate",0.2)),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(0.001), loss=allg.get("loss_function","binary_crossentropy"), metrics=["accuracy"])
    return model

model_folder = allg["model_folder"]
os.makedirs(model_folder, exist_ok=True)
model_path = os.path.join(model_folder, allg["use_model_file"])
json_path = model_path.replace(".keras",".json")

# Lade Modell
if os.path.exists(model_path):
    model = load_model(model_path)
    print(f"Model geladen: {model_path}")
    # Lade Transaktionen
    if os.path.exists(json_path):
        with open(json_path,"r") as f:
            model_data = json.load(f)
        balance = model_data.get("balance", balance_settings["initial_balance"])
        transactions = model_data.get("transactions", [])
    else:
        balance = balance_settings["initial_balance"]
        transactions = []
else:
    model = create_model()
    balance = balance_settings["initial_balance"]
    transactions = []
    print("Neues Modell erstellt.")

# Scaler
scaler = StandardScaler()

ValueError: File format not supported: filepath=models\. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(models\, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
# -------------------------------
# Funktion zum Speichern der Transaktionen
# -------------------------------
def save_model_json(balance, transactions, model_path):
    json_path = os.path.join(
        allg["model_folder"], 
        f"{os.path.splitext(os.path.basename(model_path))[0]}_transactions.json"
    )
    
    data = {
        "model": os.path.basename(model_path),
        "created": str(datetime.now()),
        "balance": balance,
        "transactions": transactions
    }

    with open(json_path,"w") as f:
        json.dump(data, f, indent=4)
    
    print(f"Transactions JSON gespeichert → {json_path}")
    return json_path


In [ ]:
# -------------------------------
# Offline Training & Trade Logging
# -------------------------------
def offline_train(csv_file):
    df = pd.read_csv(csv_file)

    # Features + Label
    X = df[["open","high","low","close","prev_close","current_close","volume"]].values
    y = (df["color"].str.lower() == "green").astype(int).values

    # Scaling
    X = scaler.fit_transform(X)

    # Balance initialisieren
    balance = balance_settings.get("initial_balance", 1000)
    transactions = []

    for idx,row in df.iterrows():
        input_X = np.array([X[idx]])
        y_pred_prob = model.predict(input_X, verbose=0).flatten()[0]
        pred_label = "green" if y_pred_prob>0.5 else "red"
        true_label = "green" if row["color"].lower() == "green" else "red"

        # Trade-Logik: Investieren
        invested_balance = min(balance, 100)  # Beispiel: max 100 pro Trade
        entry_price = row["prev_close"]
        exit_price = row["close"]

        # Profitberechnung: long wenn prediction green, short wenn red
        profit = (exit_price - entry_price) * (1 if pred_label=="green" else -1)
        profit_percent = (profit / invested_balance) * 100 if invested_balance>0 else 0
        balance += profit

        transactions.append({
            "entry_time": str(row["timestamp"]),
            "exit_time": str(row["timestamp"]),
            "symbol": row.get("symbol","BTCUSDT"),
            "invested_balance": invested_balance,
            "profit_percent": profit_percent,
            "balance_after": balance,
            "predicted": pred_label,
            "actual": true_label,
            "profit": float(profit),
            "confidence": {"green": float(y_pred_prob), "red": float(1-y_pred_prob)}
        })

        # Optional: eine Epoche Training auf einzelne Candle
        model.fit(input_X, np.array([1 if true_label=="green" else 0]), epochs=1, verbose=0)

    # Modell speichern
    model.save(model_path)
    print(f"Offline Training abgeschlossen → {model_path}")

    # JSON speichern
    json_path = os.path.join(allg["model_folder"], f"{os.path.splitext(allg['use_model_file'])[0]}_transactions.json")
    with open(json_path, "w") as f:
        json.dump({
            "model": os.path.basename(model_path),
            "created": str(datetime.now()),
            "balance": balance,
            "transactions": transactions
        }, f, indent=4)
    print(f"Transactions JSON gespeichert → {json_path}")

    # CSV speichern
    csv_path = os.path.join(allg["model_folder"], f"{os.path.splitext(allg['use_model_file'])[0]}_transactions.csv")
    pd.DataFrame(transactions).to_csv(csv_path, index=False)
    print(f"Transactions CSV gespeichert → {csv_path}")

In [ ]:
# -------------------------------
# CSV-Export für Analyse
# -------------------------------
def export_transactions_csv(csv_file=None):
    if csv_file is None:
        csv_file = json_path.replace(".json","_transactions.csv")
    pd.DataFrame(transactions).to_csv(csv_file,index=False)
    print(f"Transaktionen exportiert → {csv_file}")

In [ ]:
# -------------------------------
# Funktion zum Speichern der Transaktionen (JSON + CSV)
# -------------------------------
def save_transactions(balance, transactions, model_path):
    base_name = os.path.splitext(os.path.basename(model_path))[0]
    
    # JSON speichern
    json_path = os.path.join(allg["model_folder"], f"{base_name}_transactions.json")
    data = {
        "model": os.path.basename(model_path),
        "created": str(datetime.now()),
        "balance": balance,
        "transactions": transactions
    }
    with open(json_path, "w") as f:
        json.dump(data, f, indent=4)
    
    # CSV speichern
    csv_path = os.path.join(allg["model_folder"], f"{base_name}_transactions.csv")
    
    # CSV Header
    csv_columns = [
        "timestamp", "symbol", "open", "high", "low", "close", 
        "prev_close", "current_close", "volume", 
        "balance_invested", "profit", "predicted", "actual", 
        "confidence_green", "confidence_red"
    ]
    
    csv_rows = []
    for t in transactions:
        csv_rows.append({
            "timestamp": t.get("timestamp"),
            "symbol": t.get("symbol"),
            "open": t.get("open"),
            "high": t.get("high"),
            "low": t.get("low"),
            "close": t.get("close"),
            "prev_close": t.get("prev_close"),
            "current_close": t.get("current_close"),
            "volume": t.get("volume"),
            "balance_invested": t.get("balance_invested", 0),
            "profit": t.get("profit", 0),
            "predicted": t.get("predicted"),
            "actual": t.get("actual"),
            "confidence_green": t.get("confidence", {}).get("green", 0),
            "confidence_red": t.get("confidence", {}).get("red", 0)
        })
    
    df_csv = pd.DataFrame(csv_rows, columns=csv_columns)
    df_csv.to_csv(csv_path, index=False)
    
    print(f"Transactions JSON gespeichert → {json_path}")
    print(f"Transactions CSV gespeichert → {csv_path}")
    
    return json_path, csv_path


In [ ]:
def fetch_live_data(symbol="BTCUSDT", interval="1m"):
    url = f"http://127.0.0.1:5000/api/live?symbols={symbol}&source=binance&interval={interval}"
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        if symbol in data:
            # Single candle in einer Liste zurückgeben für Konsistenz mit Train-Data
            return [data[symbol]]
        else:
            print(f"Symbol {symbol} nicht gefunden in API-Response")
            return []
    else:
        print("Fehler beim Abrufen der Live-Daten")
        return []

def fetch_train_data(symbol="BTCUSDT", interval="1m", period="7d"):
    url = f"http://127.0.0.1:5000/api/train_mode?symbols={symbol}&source=binance&interval={interval}&period={period}"
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        if symbol in data and "history" in data[symbol]:
            return data[symbol]["history"]
        else:
            print(f"Keine Historie gefunden für Symbol {symbol}")
            return []
    else:
        print("Fehler beim Abrufen der Trainings-Daten")
        return []


In [ ]:
# Globale Variablen
scaler = None
model = None  # Dein trainiertes Keras/TensorFlow-Modell
allg = {"feature_scaling": True}  # Features skalieren
training_options = {"punish_on_wrong": True}  # Strafe für falsche Vorhersagen

# -------------------------------
# Fit Scaler auf Trainingsdaten (einmalig)
# -------------------------------
def fit_scaler_on_training_data(candle_data):
    global scaler
    if allg.get("feature_scaling", False):
        features = np.array([[c["open"], c["high"], c["low"], c["close"],
                              c["prev_close"], c["current_close"], c["volume"]]
                             for c in candle_data])
        scaler.fit(features)
        print("Scaler erfolgreich gefittet!")
    else:
        print("Feature Scaling ist deaktiviert, Scaler nicht benötigt.")

# -------------------------------
# Prepare Transactions
# -------------------------------
def prepare_transactions(candle_data, balance, train_mode=False, invest_pct=0.1):
    global scaler, model
    transactions = []

    if train_mode:
        # Beim Offline-Training sollte der Scaler schon gefittet sein
        if allg.get("feature_scaling", False) and scaler is None:
            fit_scaler_on_training_data(candle_data)

    for candle in candle_data:
        invested = balance * invest_pct

        # Feature-Vektor
        features = np.array([[candle["open"], candle["high"], candle["low"], candle["close"],
                              candle["prev_close"], candle["current_close"], candle["volume"]]])

        # Feature Scaling
        if allg.get("feature_scaling", False):
            if scaler is None:
                raise ValueError("Scaler ist nicht gefittet! Bitte zuerst fit_scaler_on_training_data aufrufen.")
            features_scaled = scaler.transform(features)
        else:
            features_scaled = features

        # Vorhersage
        if not train_mode:
            y_pred_prob = model.predict(features_scaled, verbose=0).flatten()[0]
            pred_label = "green" if y_pred_prob > 0.5 else "red"
        else:
            y_pred_prob = 0.7  # Dummy für Offline
            pred_label = "green" if y_pred_prob > 0.5 else "red"

        actual_label = "green" if candle["color"].lower() == "green" else "red"

        # Profitberechnung
        entry_price = candle["prev_close"]
        exit_price = candle["close"]
        direction = 1 if pred_label == "green" else -1
        profit = (exit_price - entry_price) * direction
        profit_percent = (profit / invested) * 100 if invested > 0 else 0

        # Optional Strafe bei falscher Vorhersage
        if train_mode and training_options.get("punish_on_wrong", False):
            if pred_label != actual_label:
                profit *= 0.0

        balance += profit

        # Offline Training für jede Candle
        if train_mode:
            model.fit(features_scaled, np.array([1 if actual_label=="green" else 0]), epochs=1, verbose=0)

        confidence = {"green": float(y_pred_prob), "red": float(1 - y_pred_prob)}

        transactions.append({
            "timestamp": candle["timestamp"],
            "symbol": candle.get("symbol", "BTCUSDT"),
            "open": candle["open"],
            "high": candle["high"],
            "low": candle["low"],
            "close": candle["close"],
            "prev_close": candle["prev_close"],
            "current_close": candle["current_close"],
            "volume": candle["volume"],
            "balance_invested": invested,
            "profit": float(profit),
            "profit_percent": float(profit_percent),
            "balance_after": balance,
            "predicted": pred_label,
            "actual": actual_label,
            "confidence": confidence
        })

    return transactions, balance

In [ ]:
# -------------------------------
# Vorbereitung: Scaler fitten
# -------------------------------
def fit_scaler_on_training_data(train_candles):
    global scaler
    if allg.get("feature_scaling", False):
        features_train = np.array([[c["open"], c["high"], c["low"], c["close"],
                                    c["prev_close"], c["current_close"], c["volume"]]
                                   for c in train_candles])
        scaler.fit(features_train)
        print("Scaler erfolgreich gefittet.")
    else:
        print("Feature Scaling ist deaktiviert. Kein Scaler benötigt.")

# -------------------------------
# Haupt-Workflow: Training + Live
# -------------------------------
def run_trading_pipeline(initial_balance, period="7d"):
    global scaler, model
    
    balance = initial_balance
    
    # 1️⃣ Trainingsdaten abrufen
    train_candles = fetch_train_data(period=period)
    
    # 2️⃣ Scaler fitten
    fit_scaler_on_training_data(train_candles)
    
    # 3️⃣ Offline-Training durchführen
    transactions_train, balance = prepare_transactions(train_candles, balance, train_mode=True)
    print(f"Offline-Training abgeschlossen. Balance nach Training: {balance:.2f}")
    
    # 4️⃣ Live-Daten abrufen
    live_candles = fetch_live_data()
    
    # 5️⃣ Live-Transaktionen vorbereiten (nur Analyse, Balance optional)
    transactions_live, balance_live = prepare_transactions(live_candles, balance, train_mode=False)
    print(f"Live-Analyse abgeschlossen. Live-Balance: {balance_live:.2f}")
    
    # 6️⃣ Speichern
    json_path, csv_path = save_transactions(balance_live, transactions_live, model_path)
    print(f"Transaktionen gespeichert: JSON={json_path}, CSV={csv_path}")
    
    return transactions_train, transactions_live, balance, balance_live


In [ ]:
# Startbalance
balance = balance_settings["initial_balance"]

# 1️⃣ Trainingsdaten abrufen
train_candles = fetch_train_data(period="7d")

# 2️⃣ Scaler fitten
fit_scaler_on_training_data(train_candles)

# 3️⃣ Offline-Training: train_mode=True
transactions_train, balance_train = prepare_transactions(train_candles, balance, train_mode=True)

# 4️⃣ Live-Daten abrufen
live_candles = fetch_live_data()

# 5️⃣ Live-Analyse: train_mode=False, eigene Balance
transactions_live, balance_live = prepare_transactions(live_candles, balance, train_mode=False)
